## Импорт библиотек

In [1]:
import pandas as pd

## Обзор данных

#### Чтение датасета

In [2]:
df = pd.read_csv("airlines_dataset.csv")

In [9]:
df.head(2)

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,400740,XE,3036,IAH,CHS,4.0,1195,131.0,0
1,194808,UA,315,DEN,ONT,7.0,707,145.0,0


#### Обзор столбцов датасета

- `id` - номер вылета
- `Airline` - страна, которой принадлежит борт
- `Flight` - номер борта
- `AirportFrom` - аэропорт отправления
- `AirportTo` - аэропорт прибытия
- `DayOfWeek` - день недели
- `Time` - общее время налёта
- `Length` - общее расстояние полётов
- `Delay`- статус задержки

#### Получение информации о данных в столбцах датафрейма

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           400000 non-null  int64  
 1   Airline      399264 non-null  object 
 2   Flight       400000 non-null  int64  
 3   AirportFrom  400000 non-null  object 
 4   AirportTo    398843 non-null  object 
 5   DayOfWeek    399673 non-null  float64
 6   Time         400000 non-null  int64  
 7   Length       399517 non-null  float64
 8   Delay        400000 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 27.5+ MB


#### Выводы

- Заголовки нужно привести к lower case и заменить разделение заглавными буквами на знаки нижнего подчёркивания
- Требуется обработка пропущенных значений
- Требуется обработка дубликатов
- Требуется удалить столбец id
- Требуется привести столбцы `Airline`, `AirportFrom` и `AirportTo` к числовому формату 

## Предварительная обработка датасета